In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive/') # this makes your Drive appear in 'content' folder
df=pd.read_csv('/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/test.csv',
                         index_col=0)
print(df.size)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


<ipython-input-13-8b80872d1773>:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/test.csv',


24655512


dense columns: rating_y,loves_count,reviews

categorical columns: everything else

label:rating_x

In [14]:
print(df.columns)

Index(['skin_tone_quartile_num', 'Unnamed: 1', 'Unnamed: 0', 'author_id',
       'rating_x', 'is_recommended', 'helpfulness', 'total_feedback_count',
       'total_neg_feedback_count', 'total_pos_feedback_count', 'skin_type',
       'hair_color', 'product_id', 'skin_tone_num', 'eye_color_num',
       'skin_type_num', 'hair_color_num', 'review_text_processed',
       'nltk_analysis', 'skin_tone_quartile_num.1', 'product_name', 'brand_id',
       'brand_name', 'loves_count', 'rating_y', 'reviews', 'limited_edition',
       'new', 'online_only', 'out_of_stock', 'sephora_exclusive', 'Award',
       'Clean at Sephora', 'combined_category_Bath & Body',
       'combined_category_Fragrance', 'combined_category_Gifts',
       'combined_category_Hair', 'combined_category_Makeup',
       'combined_category_Men', 'combined_category_Skincare',
       'combined_category_Tools & Brushes',
       'combined_category_Value & Gift Sets', 'Ascorbyl Glucoside', 'Retinol',
       'Ceramide', 'Lactic acid', 

In [15]:
import re
df["nltk_analysis_pos"]=df["nltk_analysis"].apply(lambda x: float(re.search(r"'pos': (\d+\.\d+)", x).group(1)))
df["nltk_analysis_neu"]=df["nltk_analysis"].apply(lambda x: float(re.search(r"'neu': (\d+\.\d+)", x).group(1)))

In [16]:
print(df['nltk_analysis_pos'])
df["label"]=df['rating_x']
df["label"]=np.where(df["label"]==5,1,0)
print(df['label'].nunique())
df=df.drop(labels=["rating_x"],axis=1)

0         0.449
1         0.333
2         0.587
3         0.476
4         0.370
          ...  
440272    0.317
440273    0.373
440274    0.350
440275    0.506
440276    0.437
Name: nltk_analysis_pos, Length: 440277, dtype: float64
2


In [17]:
df=df.drop(labels=["author_id","review_text_processed","Unnamed: 1","Unnamed: 0",'is_recommended','helpfulness','total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count','product_id'],axis=1)

column_list=df.columns.tolist()
df.size

21133296

In [18]:
dense_cols=["rating_y","loves_count","reviews"]
rm_list=["rating_y","loves_count","reviews"]

for ele in rm_list:
  print(ele)
  column_list.remove(ele)

cat_cols=column_list
print(df.size)
df.dropna()
df.columns

rating_y
loves_count
reviews
21133296


Index(['skin_tone_quartile_num', 'skin_type', 'hair_color', 'skin_tone_num',
       'eye_color_num', 'skin_type_num', 'hair_color_num', 'nltk_analysis',
       'skin_tone_quartile_num.1', 'product_name', 'brand_id', 'brand_name',
       'loves_count', 'rating_y', 'reviews', 'limited_edition', 'new',
       'online_only', 'out_of_stock', 'sephora_exclusive', 'Award',
       'Clean at Sephora', 'combined_category_Bath & Body',
       'combined_category_Fragrance', 'combined_category_Gifts',
       'combined_category_Hair', 'combined_category_Makeup',
       'combined_category_Men', 'combined_category_Skincare',
       'combined_category_Tools & Brushes',
       'combined_category_Value & Gift Sets', 'Ascorbyl Glucoside', 'Retinol',
       'Ceramide', 'Lactic acid', 'Squalane', 'Aloe', 'Salicylic Acid', 'Shea',
       'Vitamin E/tocopherols', 'jojoba', 'peppermint', 'honey/miel',
       'Phenoxyethanol', 'Rose', 'nltk_analysis_pos', 'nltk_analysis_neu',
       'label'],
      dtype='objec

In [19]:
df.size

21133296

In [20]:
from sklearn import preprocessing
df[dense_cols] = preprocessing.StandardScaler().fit_transform(df[dense_cols])
mappings = {
    col: dict(zip(values, range(len(values))))
    for col, values in map(lambda col: (col, df[col].unique()), cat_cols)
}
for col, mapping in mappings.items():
    df[col] = df[col].map(mapping.get)

In [21]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [22]:
emb_counts = [len(df[c].unique()) for c in cat_cols]

In [23]:
TRAIN_SPLIT = 0.2
VALIDATION_SPLIT = 0.5

ds = tf.data.Dataset.zip((
    tf.data.Dataset.from_tensor_slices((
        tf.cast(df[dense_cols].values, tf.float32),
        tf.cast(df[cat_cols].values, tf.int32),
    )),
    tf.data.Dataset.from_tensor_slices((
        tf.cast(to_categorical(df['label'].values, num_classes=2), tf.float32)
    ))
)).shuffle(buffer_size=2048)


ds_test = ds.take(int(len(ds) * TRAIN_SPLIT))
ds_train = ds.skip(len(ds_test))
ds_valid = ds_test.take(int(len(ds_test) * VALIDATION_SPLIT))
ds_test = ds_test.skip(len(ds_valid))
print(ds)

<_ShuffleDataset element_spec=((TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(45,), dtype=tf.int32, name=None)), TensorSpec(shape=(2,), dtype=tf.float32, name=None))>


In [24]:
def MLP(arch, activation='relu', out_activation=None):
    mlp = tf.keras.Sequential()

    for units in arch[:-1]:
        mlp.add(tf.keras.layers.Dense(units, activation=activation))

    mlp.add(tf.keras.layers.Dense(arch[-1], activation=out_activation))

    return mlp


class SecondOrderFeatureInteraction(tf.keras.layers.Layer):
    def __init__(self, self_interaction=False):
        super(SecondOrderFeatureInteraction, self).__init__()
        self.self_interaction = self_interaction

    def call(self, inputs):
        batch_size = tf.shape(inputs[0])[0]
        concat_features = tf.stack(inputs, axis=1)

        dot_products = tf.matmul(concat_features, concat_features, transpose_b=True)

        ones = tf.ones_like(dot_products)
        mask = tf.linalg.band_part(ones, 0, -1)
        out_dim = int(len(inputs) * (len(inputs) + 1) / 2)

        if not self.self_interaction:
            mask = mask - tf.linalg.band_part(ones, 0, 0)
            out_dim = int(len(inputs) * (len(inputs) - 1) / 2)

        flat_interactions = tf.reshape(tf.boolean_mask(dot_products, mask), (batch_size, out_dim))
        return flat_interactions


class DLRM(tf.keras.Model):
    def __init__(
            self,
            embedding_sizes,
            embedding_dim,
            arch_bot,
            arch_top,
            self_interaction,
    ):
        super(DLRM, self).__init__()
        self.emb = [tf.keras.layers.Embedding(size, embedding_dim) for size in embedding_sizes]
        self.bot_nn = MLP(arch_bot, out_activation='relu')
        self.top_nn = MLP(arch_top, out_activation='sigmoid')
        self.interaction_op = SecondOrderFeatureInteraction(self_interaction)

    def call(self, input):
        input_dense, input_cat = input
        emb_x = [E(x) for E, x in zip(self.emb, tf.unstack(input_cat, axis=1))]
        dense_x = self.bot_nn(input_dense)

        Z = self.interaction_op(emb_x + [dense_x])
        z = tf.concat([dense_x, Z], axis=1)
        p = self.top_nn(z)

        return p

In [25]:
model = DLRM(
    embedding_sizes=emb_counts,
    embedding_dim=2,
    arch_bot=[8, 2],
    arch_top=[128, 64, 2],
    self_interaction=False
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
label_counts = df.groupby('label')['skin_tone_num'].count()
print(f'Baseline: {max(label_counts.values) / sum(label_counts.values) * 100}%')

Baseline: 86.41582458316014%


In [ ]:
BATCH_SIZE = 32

history =model.fit(
    ds_train.batch(BATCH_SIZE),
    validation_data=ds_valid.batch(BATCH_SIZE),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
    ],
    epochs=20,
    verbose=1,
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


11007/11007 [==============================] - 222s 19ms/step - loss: 0.3912 - accuracy: 0.8652 - val_loss: 0.4515 - val_accuracy: 0.8604
Epoch 2/20
11007/11007 [==============================] - 193s 17ms/step - loss: 0.3728 - accuracy: 0.8681 - val_loss: 0.5568 - val_accuracy: 0.8350
Epoch 3/20
11007/11007 [==============================] - 195s 18ms/step - loss: 0.3564 - accuracy: 0.8725 - val_loss: 0.5842 - val_accuracy: 0.8307
Epoch 4/20
11007/11007 [==============================] - 189s 17ms/step - loss: 0.3451 - accuracy: 0.8751 - val_loss: 0.5747 - val_accuracy: 0.8431
Epoch 5/20
11007/11007 [==============================] - 188s 17ms/step - loss: 0.3367 - accuracy: 0.8770 - val_loss: 0.5529 - val_accuracy: 0.8470
Epoch 6/20
11007/11007 [==============================] - 192s 17ms/step - loss: 0.3301 - accuracy: 0.8782 - val_loss: 0.5189 - val_accuracy: 0.8514
Epoch 7/20
11007/11007 [==============================] - 189s 17ms/step - loss: 0.3245 - accuracy: 0.8792 - val_loss

In [ ]:
model.summary()


Model: "dlrm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  8         
                                                                 
 embedding_1 (Embedding)     multiple                  8         
                                                                 
 embedding_2 (Embedding)     multiple                  14        
                                                                 
 embedding_3 (Embedding)     multiple                  26        
                                                                 
 embedding_4 (Embedding)     multiple                  10        
                                                                 
 embedding_5 (Embedding)     multiple                  8         
                                                                 
 embedding_6 (Embedding)     multiple                  14     

In [ ]:
results = model.evaluate(ds_test.batch(BATCH_SIZE))
print(f'Loss {results[0]}, Accuracy {results[1]}')

1376/1376 [==============================] - 6s 4ms/step - loss: 0.4553 - accuracy: 0.8585
Loss 0.4553130567073822, Accuracy 0.8585218787193298


In [ ]:
model.save('/content/drive/MyDrive/bttai_accenture/Data Cleaning/rongzi/saved_model/my_model')
#model.predict_proba(ds_test.batch(BATCH_SIZE))
#model.save('saved_model/my_model')

In [ ]:
res=model.predict(ds_test.batch(BATCH_SIZE))

1376/1376 [==============================] - 11s 7ms/step


In [ ]:
print(res)

[[0.7667134  0.23557074]
 [0.6406712  0.35949257]
 [0.93730134 0.06182032]
 ...
 [0.9093479  0.09050626]
 [0.9136635  0.08672784]
 [0.97511613 0.02364872]]
